In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import World
from src import Sphere, Plane
from src import PointLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src import NormalShader

In [ ]:
# colors
brown = Color.custom_rgb(150, 75, 0)
red = Color.custom_rgb(255, 0, 0)

# material
glossy_red =  PhongMaterial(name="glossy_red", base_color=red, spec_color=Color.custom_rgb(255, 255, 255), shininess=100, ior=1.0, reflectivity=0.3, transparency=0.0)
mate_blue = PhongMaterial(name="mate_blue", base_color=Color.custom_rgb(0, 0, 255), spec_color=Color.custom_rgb(200, 200, 200), shininess=10, ior=1.3, reflectivity=0.1, transparency=0.0)

# objects
sphere = Sphere(center=Vertex(0, 0.2, -5), radius=1.0, material=glossy_red)
plane = Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=mate_blue)

# world
world = World()
world.add(sphere)
world.add(plane)

# lights
point_light = PointLight(position=Vertex(5, 5, 0), intensity=2000.0, falloff=0.01)

# scene setup
camera = Camera(
    fov = 40,
    resolution = Resolution.R360p,
    origin = Vertex(0, 0, 0),
    direction = Vector(0, 0, -1),
)

scene = Scene(
    camera = camera,
    world = world,
    lights = [point_light],
    skybox_path = None
)

scene.validate()

# TODO write custom shader that shows normals as colors

In [ ]:
norm_shader = NormalShader()
scene.render()